In [1]:
import re
import time
import requests
import selenium
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from sqlalchemy import create_engine, text
from dbio import load_data, to_db

root


In [2]:
#!pip install pymysql

# 1. 핀테크 지원센터 홈페이지 -> 알림마당 > 핀테크 정보 > 핀테크 뉴스에 접속

In [3]:
params = dict(board_id=6,menu_id=6500,miv_pageNo=1)
url = "https://fintech.or.kr/web/board/boardContentsListPage.do"
r = requests.get(url, params=params)
print(r.url)
soup = bs(r.text,'html')

https://fintech.or.kr/web/board/boardContentsListPage.do?board_id=6&menu_id=6500&miv_pageNo=1


# 2. 일자별로 있는 [11/18 핀테크 주요뉴스] 안에 있는 뉴스 제목과 원문 link 수집 

In [4]:
2. 일자별로 있는 [11/18 핀테크 주요뉴스] 안에 있는 뉴스 제목과 원문 link 수집 Xparams = dict(miv_pageNo=1,mode='W',board_id=6,searchkey='I')
url = "https://fintech.or.kr/web/board/boardContentsList.do"
r = requests.get(url, params=params)
print(r.url)
soup = bs(r.text,'html')

https://fintech.or.kr/web/board/boardContentsList.do?miv_pageNo=1&mode=W&board_id=6&searchkey=I


In [5]:
soup.select_one(".boardlist  .txtl > a").get('href')

"javascript:contentsView('bf1b44ca61d74074a455cf21e180451c')"

In [6]:
soup.select_one(".boardlist  .txtl > a").text

'[경기도] 미래콘텐츠 성과공유회(AXR)에 초대합니다. (11/20(목) ~ 21(금))'

In [7]:
# import requests

# api_url = "https://fintech.or.kr/web/user/homepageMenuList.do?up_menu_id=6000"
# res = requests.get(api_url)
# print(res.json())

In [8]:
news_list = []


options = Options()
options.add_experimental_option("detach", True)
options.add_argument("start-maximized")
options.add_argument("Chrome/141.0.0.0")
options.add_argument("lang=ko_KR")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
    )


for num in range(1,20):
    params = dict(miv_pageNo=f"{num}",mode='W',board_id=6,searchkey='I')
    url = "https://fintech.or.kr/web/board/boardContentsList.do"
    r = requests.get(url, params=params)
#    print(r.url)
    soup = bs(r.text,'html')
    
    #웹브라우저에서 url열기
    driver.get(r.url)
    
    news_list.append(soup.select("a.txtl"))
    

In [9]:
news_text = []
full_href = []

for paige in news_list:
    for element in paige:
        if "핀테크 주요뉴스" in element.text:
            news_text.append(element.text.strip())
            full_href.append(element.get('href'))
        else:
            pass


In [10]:
title = []
for i in news_text[::]:
    i = i.replace("[","").replace("]","")
    title.append(i)
title

['11/20 핀테크 주요뉴스',
 '11/19 핀테크 주요뉴스',
 '11/18 핀테크 주요뉴스',
 '11/17 핀테크 주요뉴스',
 '11/14 핀테크 주요뉴스',
 '11/13 핀테크 주요뉴스',
 '11/12 핀테크 주요뉴스',
 '11/11 핀테크 주요뉴스',
 '11/10 핀테크 주요뉴스',
 '11/07 핀테크 주요뉴스',
 '11/06 핀테크 주요뉴스',
 '11/05 핀테크 주요뉴스',
 '11/04 핀테크 주요뉴스',
 '11/03 핀테크 주요뉴스',
 '10/31 핀테크 주요뉴스',
 '10/30 핀테크 주요뉴스',
 '10/29 핀테크 주요뉴스',
 '10/28 핀테크 주요뉴스',
 '10/27 핀테크 주요뉴스',
 '10/24 핀테크 주요뉴스',
 '10/23 핀테크 주요뉴스',
 '10/22 핀테크 주요뉴스',
 '10/21 핀테크 주요뉴스',
 '10/20 핀테크 주요뉴스',
 '10/17 핀테크 주요뉴스',
 '10/16 핀테크 주요뉴스',
 '10/15 핀테크 주요뉴스',
 '10/14 핀테크 주요뉴스',
 '10/13 핀테크 주요뉴스',
 '10/10 핀테크 주요뉴스',
 '10/02 핀테크 주요뉴스',
 '10/01 핀테크 주요뉴스',
 '9/30 핀테크 주요뉴스',
 '9/29 핀테크 주요뉴스',
 '9/26 핀테크 주요뉴스',
 '9/25 핀테크 주요뉴스',
 '9/24 핀테크 주요뉴스',
 '9/23 핀테크 주요뉴스',
 '9/22 핀테크 주요뉴스',
 '9/19 핀테크 주요뉴스',
 '9/18 핀테크 주요뉴스',
 '9/17 핀테크 주요뉴스',
 '9/16 핀테크 주요뉴스',
 '9/15 핀테크 주요뉴스',
 '9/12 핀테크 주요뉴스',
 '9/11 핀테크 주요뉴스',
 '9/10 핀테크 주요뉴스',
 '9/09 핀테크 주요뉴스',
 '9/08 핀테크 주요뉴스',
 '9/05 핀테크 주요뉴스',
 '9/04 핀테크 주요뉴스',
 '9/03 핀테크 주요뉴스',
 '9/02 핀테크 주요뉴스',
 '9/01 핀테크 주요뉴

In [11]:
code = []
for c in full_href[::]:
    c = c.replace("javascript:contentsView('","").replace("')","")
    code.append(c)
code

['12c003b7478b4c359c558ecc1edcc342',
 'e5bc59a8918247a6943cc881219e5bec',
 '0579e2d50ca4449f97f5c21e6acf3d1c',
 'bc4b472316354894a61a9d7515fd585a',
 '06baae1d74e84020897b429e5908b67d',
 '4344b8eae1af467ab79269e3ccbe8727',
 'f6d1b17da67748bbbd638a8498cab027',
 '65c5f16779c34033b575a702bad9ea15',
 '29a8c0a2afff4200b3846431fd3daeb8',
 '4bf5be7d0aff47bdabbd34bc2962ab50',
 '0fa0423c31b2439ba94e894e3897aada',
 'b08264a2396f4bd1aea6b1632ade7919',
 '0c07345c3b6149a983fd78ef5487ae96',
 '50f770f84cd44d078133ba3869256a22',
 '4383ccafc29440d885a80d9daef6f134',
 'f4a8b125276d4d49bfeb7dcf711907ee',
 'fcde75e18f004f5eb78b2de07feefad5',
 '0ff8cc250a7942339d1a6a6c15a929d3',
 '8c6520f7153c438bbb38247502fdb72a',
 'e758d4898a944cd794174e35146ff1b9',
 'c59f6515c85e4df9a45b54987adeb7a5',
 '7fcecb46bde04842878b8dee94a21350',
 'f61663288d694d068e33d00054b22595',
 '760c4ae4b76f449aa23de004e8200bc5',
 'ad2eb2c521ab4e9fa32cb00a625cbd69',
 'c80032073f994fb4b669d70bb5917c18',
 'c579f3baf033478483098dc813b47e1f',
 

In [12]:
fir = {
    '코드날짜':title,
    '코드':code
}
df = pd.DataFrame(fir)
df.iloc[:-11, :]

,코드날짜,코드
0,11/20 핀테크 주요뉴스,12c003b7478b4c359c558ecc1edcc342
1,11/19 핀테크 주요뉴스,e5bc59a8918247a6943cc881219e5bec
2,11/18 핀테크 주요뉴스,0579e2d50ca4449f97f5c21e6acf3d1c
3,11/17 핀테크 주요뉴스,bc4b472316354894a61a9d7515fd585a
4,11/14 핀테크 주요뉴스,06baae1d74e84020897b429e5908b67d
...,...,...
209,1/8 핀테크 주요뉴스,738ae2219ca2474f9c9a596ff3645440
210,1/7 핀테크 주요뉴스,39992a0f6a9449f4918a10a3c04242f8
211,1/6 핀테크 주요뉴스,0da651f76e714d31882ae396db7e2e32
212,1/3 핀테크 주요뉴스,2801b7e664ae44cc885dc59d3e7262d1


In [13]:
# 날짜별 핀테크 뉴스 수집
all_news = []
all_date = []
for num2 in code:
    url2 = "https://fintech.or.kr/web/board/boardContentsView.do"
    payload2= dict(miv_pageNo=1,contents_id=f"{num2}",board_id=6)
    r2 = requests.get(url2, params=payload2)
    print(r2.url)
    soup2 = bs(r2.text,'lxml')
    date = soup2.select_one("div > div.board_detail > div.title_box > ul > li").text.replace("작성일 : ","")
    all_date.append(date)
    all_news.append(soup2.select("tbody  td > a"))

https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=12c003b7478b4c359c558ecc1edcc342&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=e5bc59a8918247a6943cc881219e5bec&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=0579e2d50ca4449f97f5c21e6acf3d1c&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=bc4b472316354894a61a9d7515fd585a&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=06baae1d74e84020897b429e5908b67d&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=4344b8eae1af467ab79269e3ccbe8727&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=f6d1b17da67748bbbd638a8498cab027&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=65c5f16779c34033b575a702bad9ea15&board_id=6
https://fintech.or.kr/we

https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=85c546f07b994f7695f7888753b1b0c6&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=9b3a489dc32a418bbc30e9c0b2eaf1f4&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=440d49ca27fb4689acf474a9b2d18572&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=84ff97db81644462a985b4df92211fa2&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=cf3e7c0972914a938495faf2eac91af2&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=eca31baf0fea4a199384577f91cad131&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=faf4f7bbdf1b481287bbe11a937493a0&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=0a92d98989e94bc393faea269685d462&board_id=6
https://fintech.or.kr/we

https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=17f72c8775714d47ba0fb664a977c40d&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=f80f1835feb94cdd9b6614df3c906d2a&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=3699d6761090427b9f335d9b66282ca4&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=df08918476ea4fc689a72fa8a307e308&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=2c2c54e18dd34dd8aba82949adbd5f2c&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=8d01185e17b748bf9dfec8da960ad913&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=cc688a8f1e8f4e10a1acf485e5709ed9&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=28d70db306e8457daea0f4162145d574&board_id=6
https://fintech.or.kr/we

https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=e5727dfedfa540bda6f83914a08c95cf&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=a3f748fee1f34b1eba481d9a7938d922&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=929aa5e1145149028e72984bdfd6a5b7&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=865734926c364ac8b8440eb866449d6f&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=738ae2219ca2474f9c9a596ff3645440&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=39992a0f6a9449f4918a10a3c04242f8&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=0da651f76e714d31882ae396db7e2e32&board_id=6
https://fintech.or.kr/web/board/boardContentsView.do?miv_pageNo=1&contents_id=2801b7e664ae44cc885dc59d3e7262d1&board_id=6
https://fintech.or.kr/we

# 3. 원문링크로 들어가서 뉴스 본문을 2025년 1월 2일까지  수집 후(약 18page) 데이터 베이스에 저장 

In [15]:
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import pandas as pd

# code 리스트는 이미 있다고 가정
# code = [...]

final_news_link = []
final_news_date = []
final_news_title = []
final_news_text = []

for num2 in tqdm(code, desc="뉴스 링크·날짜·제목 수집 중"):
    url2 = "https://fintech.or.kr/web/board/boardContentsView.do"
    payload2 = dict(miv_pageNo=1, contents_id=f"{num2}", board_id=6)

    try:
        r2 = requests.get(url2, params=payload2, timeout=5)
        r2.raise_for_status()
        soup2 = bs(r2.text, 'lxml')

        # 1) 날짜
        date_tag = soup2.select_one("div > div.board_detail > div.title_box > ul > li")
        date = date_tag.text.replace("작성일 : ", "").strip() if date_tag else "날짜_없음"

        # 2) 링크 & 제목
        link_items = soup2.select("tbody td > a")

        for item in link_items:
            link = item.get("href")
            title = item.text.strip()

            if link:
                final_news_link.append(link)
                final_news_date.append(date)
                final_news_title.append(title)

                # 3) 뉴스 본문
                try:
                    r3 = requests.get(link, timeout=5)
                    r3.raise_for_status()
                    news_soup = bs(r3.text, 'lxml')
                    text = news_soup.get_text(strip=True)
                except:
                    text = "텍스트_추출_실패"

                final_news_text.append(text)

    except requests.exceptions.RequestException as e:
        tqdm.write(f"⚠ 요청 실패 code {num2} | 오류: {e}")
        continue


# =======================================
# 📌 결과를 DataFrame으로 만들기
# =======================================

cols = ["날짜", "제목", "원문링크", "뉴스본문"]
values = (final_news_date, final_news_title, final_news_link, final_news_text)

result = {}
for col, val in zip(cols, values):
    result[col] = val

df2 = pd.DataFrame(result)

print("\n📌 DataFrame 생성 완료!")
print(df2.head())

# =======================================
# 📌 DB에 저장
# =======================================

to_db("2025fintech_news", "2025fintech_news", df2)

print("\n🎉 데이터베이스 저장 완료!")

뉴스 링크·날짜·제목 수집 중: 100%|█████████████████████████████████████████████████| 225/225 [1:00:27<00:00, 16.12s/it]



📌 DataFrame 생성 완료!
           날짜                                                 제목  \
0  2025-11-20                    한국핀테크지원센터, 싱가포르서 ‘K-핀테크 공동관’ 운영   
1  2025-11-20  [한국핀테크지원센터] 2025 핀테크 전문가(개발자) 과정 ‘FinBoost Aca...   
2  2025-11-20  [한국핀테크산업협회] 네트워킹 행사 '핀테크 커넥팅 데이 2025' 개최(11.26...   
3  2025-11-20  [한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생...   
4  2025-11-20  [한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) ...   

                                                원문링크  \
0  https://it.chosun.com/news/articleView.html?id...   
1  https://fintech.or.kr/web/board/boardContentsV...   
2  https://fintech.or.kr/web/board/boardContentsV...   
3  https://fintech.or.kr/web/board/boardContentsV...   
4  https://fintech.or.kr/web/board/boardContentsV...   

                                                뉴스본문  
0  한국핀테크지원센터, 싱가포르서 ‘K-핀테크 공동관’ 운영 < 일반 < 기업 < 기사...  
1  핀테크 포털 - 한국핀테크지원센터메뉴 건너띄기상단메뉴 바로가기..본문 바로가기로그인...  
2  핀테크 포털 - 한국핀테크지원센터메뉴 건너띄기상단메뉴 바로가기..본문 바로가기로그인...  
3  핀테크 포털 - 한국

In [20]:
df2['뉴스본문'][0]

'한국핀테크지원센터,\xa0싱가포르서\xa0‘K-핀테크\xa0공동관’\xa0운영 < 일반 < 기업 < 기사본문 - IT조선주요서비스 바로가기본문 바로가기매체정보 바로가기로그인 바로가기기사검색 바로가기전체서비스 바로가기전체메뉴기사검색기사검색검색닫기기사검색기사검색검색닫기로그인facebookpostyoutubeUPDATED. 2025-11-20 18:36 (목)기사검색기업일반모바일·가전방송·통신반도체·디스플레이SW·보안중공업·에너지소부장·스타트업유통·쇼핑프롭테크·부동산모빌리티자동차·모빌리티로봇·드론·항공방산게임·콘텐츠게임·인터넷메타버스·VR키덜트미디어·엔터과학·헬스과학·우주의학·정책제약·바이오파이낸스금융증권핀테크·블록체인칼럼·인터뷰칼럼기고인터뷰알림알립니다인사부음새로나왔어요마소코딩코딩에듀테크컴퓨팅·AI테크리포트GLOBAL기획·연재속보백과사전속보기업일반모바일·가전방송·통신반도체·디스플레이SW·보안중공업·에너지소부장·스타트업유통·쇼핑프롭테크·부동산모빌리티자동차·모빌리티로봇·드론·항공방산게임·콘텐츠게임·인터넷메타버스·VR키덜트미디어·엔터과학·헬스과학·우주의학·정책제약·바이오파이낸스금융증권핀테크·블록체인칼럼·인터뷰칼럼기고인터뷰마소코딩코딩에듀테크컴퓨팅·AI테크리포트알림알립니다인사부음새로나왔어요GLOBAL기업일반한국핀테크지원센터,\xa0싱가포르서\xa0‘K-핀테크\xa0공동관’\xa0운영기자명허인학 기자입력 2025.11.19 09:00SNS 기사보내기페이스북(으)로 기사보내기트위터(으)로 기사보내기카카오톡(으)로 기사보내기가가이 기사를 공유합니다페이스북(으)로 기사보내기트위터(으)로 기사보내기카카오톡(으)로 기사보내기URL복사(으)로 기사보내기닫기한국핀테크지원센터는\xa0금융위원회가 추진하는 ‘핀테크 특화 해외 진출 지원 사업’의 일환으로 국내 핀테크 기업 10곳과 함께 아시아 최대 핀테크 행사 ‘싱가포르 핀테크 페스티벌 2025(Singapore Fintech Festival 2025)’에 참가해 ‘K-핀테크 공동관’을 운영하고 해외 데모데이를 10~15일 개최했다.한국핀테

In [56]:
# 뉴스링크
# news_link = [item.get("href") for row in all_news for item in row]
# len(news_link)

In [57]:
# 뉴스 제목
# news_text = [item.text for row in all_news for item in row]
# len(news_text)

In [58]:
#  # 뉴스 텍스트
# r3 = requests.get(link, timeout=5)
# r3.raise_for_status()
# news_soup = bs(r3.text, 'lxml')
# text = news_soup.select(".nbd_table")
# text

In [59]:
# content = news_soup.select_one("#dic_area") or news_soup.select_one("div.content")
# text = content.get_text(" ", strip=True) if content else news_soup.get_text(" ", strip=True)
# text

In [60]:
# # 뉴스링크
# news_link = [item.get("href") for row in all_news for item in row]
# len(news_link)
# # 뉴스 제목
# news_text = [item.text for row in all_news for item in row]
# len(news_text)
# # 뉴스 텍스트
# news_text = []
# for index,l in enumerate(news_link):
#     print(f"인덱스: {index}, 값: {l}",end='\r')
#     r3 = requests.get(l)
#     news = bs(r3.text,'html')
#     news_text.append(news.get_text(strip=True))

# 4. 데이터 베이스에 저장된 뉴스를 데이터프레임으로 불러와서 텍스트 전처리, Wordcloud, LDA 토픽 모델링

In [2]:
sql_news_df = load_data("2025fintech_news","2025fintech_news")
sql_news_df

,날짜,제목,원문링크,뉴스본문
0,2025-11-20,"한국핀테크지원센터, 싱가포르서 ‘K-핀테크 공동관’ 운영",https://it.chosun.com/news/articleView.html?id...,"한국핀테크지원센터, 싱가포르서 ‘K-핀테크 공동관’ 운영 < 일반 < 기업 < 기사..."
1,2025-11-20,[한국핀테크지원센터] 2025 핀테크 전문가(개발자) 과정 ‘FinBoost Aca...,https://fintech.or.kr/web/board/boardContentsV...,핀테크 포털 - 한국핀테크지원센터메뉴 건너띄기상단메뉴 바로가기..본문 바로가기로그인...
2,2025-11-20,[한국핀테크산업협회] 네트워킹 행사 '핀테크 커넥팅 데이 2025' 개최(11.26...,https://fintech.or.kr/web/board/boardContentsV...,핀테크 포털 - 한국핀테크지원센터메뉴 건너띄기상단메뉴 바로가기..본문 바로가기로그인...
3,2025-11-20,[한국핀테크지원센터] (온라인) 핀테크를 통한 금융 AI 트렌드와 혁신 사례 교육생...,https://fintech.or.kr/web/board/boardContentsV...,핀테크 포털 - 한국핀테크지원센터메뉴 건너띄기상단메뉴 바로가기..본문 바로가기로그인...
4,2025-11-20,[한국핀테크지원센터] 2025년 핀테크기업 온라인 채용관 (사람인 saramin) ...,https://fintech.or.kr/web/board/boardContentsV...,핀테크 포털 - 한국핀테크지원센터메뉴 건너띄기상단메뉴 바로가기..본문 바로가기로그인...
...,...,...,...,...
11672,2024-12-16,수렁에 빠진 중국 경제... “몰락 중산층 2000만명 우버 택시로 몰렸다”,https://n.news.naver.com/mnews/article/023/000...,수렁에 빠진 중국 경제... “몰락 중산층 2000만명 우버 택시로 몰렸다”본문 바...
11673,2024-12-16,"""관세 폭탄 때문에 내년엔 더 비싸""…생필품·가전 등 사재기 열풍 부는 美",https://n.news.naver.com/mnews/article/277/000...,"""관세 폭탄 때문에 내년엔 더 비싸""…생필품·가전 등 사재기 열풍 부는 美본문 바로..."
11674,2024-12-16,"구글·메타 등과 AI칩 합작… 브로드컴, 엔비디아 대항마로",https://n.news.naver.com/mnews/article/023/000...,"구글·메타 등과 AI칩 합작… 브로드컴, 엔비디아 대항마로본문 바로가기NAVER뉴스..."
11675,2024-12-16,"美, 중국의 중동·동남아 통한 AI 반도체 우회 조달도 막는다",https://n.news.naver.com/mnews/article/023/000...,"美, 중국의 중동·동남아 통한 AI 반도체 우회 조달도 막는다본문 바로가기NAVER..."


In [26]:
# 텍스트 처리
def text_clean(x):
    # 한글 영문대소문자, 숫자만 남기고 모두 제거
    cleaned = re.sub(r'[^가-힣a-zA-Z0-9]'," ",x)
    cleaned = cleaned.replace("  "," ").replace("  "," ").strip()
    return cleaned

In [32]:
x = df2['뉴스본문'][0]

In [55]:
text_clean(x)

'한국핀테크지원센터 싱가포르서 K 핀테크 공동관 운영 일반 기업 기사본문 IT조선주요서비스 바로가기본문 바로가기매체정보 바로가기로그인 바로가기기사검색 바로가기전체서비스 바로가기전체메뉴기사검색기사검색검색닫기기사검색기사검색검색닫기로그인facebookpostyoutubeUPDATED 2025 11 20 18 36 목 기사검색기업일반모바일 가전방송 통신반도체 디스플레이SW 보안중공업 에너지소부장 스타트업유통 쇼핑프롭테크 부동산모빌리티자동차 모빌리티로봇 드론 항공방산게임 콘텐츠게임 인터넷메타버스 VR키덜트미디어 엔터과학 헬스과학 우주의학 정책제약 바이오파이낸스금융증권핀테크 블록체인칼럼 인터뷰칼럼기고인터뷰알림알립니다인사부음새로나왔어요마소코딩코딩에듀테크컴퓨팅 AI테크리포트GLOBAL기획 연재속보백과사전속보기업일반모바일 가전방송 통신반도체 디스플레이SW 보안중공업 에너지소부장 스타트업유통 쇼핑프롭테크 부동산모빌리티자동차 모빌리티로봇 드론 항공방산게임 콘텐츠게임 인터넷메타버스 VR키덜트미디어 엔터과학 헬스과학 우주의학 정책제약 바이오파이낸스금융증권핀테크 블록체인칼럼 인터뷰칼럼기고인터뷰마소코딩코딩에듀테크컴퓨팅 AI테크리포트알림알립니다인사부음새로나왔어요GLOBAL기업일반한국핀테크지원센터 싱가포르서 K 핀테크 공동관 운영기자명허인학 기자입력 2025 11 19 09 00SNS 기사보내기페이스북 으 로 기사보내기트위터 으 로 기사보내기카카오톡 으 로 기사보내기가가이 기사를 공유합니다페이스북 으 로 기사보내기트위터 으 로 기사보내기카카오톡 으 로 기사보내기URL복사 으 로 기사보내기닫기한국핀테크지원센터는 금융위원회가 추진하는 핀테크 특화 해외 진출 지원 사업 의 일환으로 국내 핀테크 기업 10곳과 함께 아시아 최대 핀테크 행사 싱가포르 핀테크 페스티벌 2025 Singapore Fintech Festival 2025 에 참가해 K 핀테크 공동관 을 운영하고 해외 데모데이를 10 15일 개최했다 한국핀테크지원센터는 싱가포르 핀테크 페스티벌 2025 Singapore Fintech Fes